In [79]:
import pandas as pd
import re

from bs4 import BeautifulSoup, SoupStrainer
import requests
import pickle

In [6]:
url = "http://insideairbnb.com/get-the-data.html"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

In [72]:
def gather_links(city,cat):

    len(links)
    links_df = pd.DataFrame(links, columns = ['link'])
    links_df2 = links_df[links_df['link'].str.contains(city)==True]
    links_df3 = links_df2[links_df2['link'].str.contains(cat)==True]
    return links_df3

In [74]:
links_sf_listings = gather_links('san-francisco','listings.csv.gz')


In [75]:
links_sf_listings['dt'] = [x.split('/')[6] for x in links_sf_listings['link']]
links_sf_listings.head()

,link,dt
7529,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7536,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7543,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7550,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7557,http://data.insideairbnb.com/united-states/ca/...,2019-01-09


In [46]:
idx = links_sf_listings['dt'].unique()
len(idx),idx

(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [60]:
def readin_gz(dt):
    url = 'http://data.insideairbnb.com/united-states/ca/san-francisco/{}/data/listings.csv.gz'.format(str(dt))

    listings = pd.read_csv(url,compression='gzip',header=0, sep=',', quotechar='"')
    return listings
    

In [64]:
listings = readin_gz(idx[0])       

In [65]:
for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i])
    print(dt)
    listings = pd.concat([listings, df], ignore_index = True,axis = 0,sort=False)
        

2019-04-03
2019-03-06
2019-02-01
2019-01-09
2018-12-06
2018-11-03
2018-10-03
2018-09-08
2018-08-06
2018-07-05
2018-05-09
2018-04-06
2018-03-04
2018-02-02
2018-01-17
2018-01-10
2017-12-07
2017-12-02
2017-11-08
2017-11-01
2017-10-02
2017-09-02
2017-08-02
2017-07-02
2017-06-02
2017-05-02
2017-04-02
2017-03-02
2017-02-02
2017-01-01
2016-12-03
2016-11-02
2016-10-01
2016-09-02
2016-08-02
2016-07-02
2016-06-02
2016-05-02
2016-04-03
2016-02-02
2015-12-02
2015-11-01
2015-09-02
2015-05-04


In [81]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,20190503153007,2019-05-03,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,...,f,f,moderate,f,f,1,1.0,0.0,0.0,1.55
1,5858,https://www.airbnb.com/rooms/5858,20190503153007,2019-05-03,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,f,f,strict_14_with_grace_period,f,f,1,1.0,0.0,0.0,0.91
2,7918,https://www.airbnb.com/rooms/7918,20190503153007,2019-05-03,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,f,f,strict_14_with_grace_period,f,f,9,0.0,9.0,0.0,0.14
3,8142,https://www.airbnb.com/rooms/8142,20190503153007,2019-05-03,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,Room rental Sunny view Rm/Wi-Fi/TV/sink/large ...,Nice and good public transportation. 7 minute...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,9,0.0,9.0,0.0,0.14
4,8339,https://www.airbnb.com/rooms/8339,20190503153007,2019-05-03,Historic Alamo Square Victorian,Pls email before booking. Interior featured i...,Please send us a quick message before booking ...,Pls email before booking. Interior featured i...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,2,2.0,0.0,0.0,0.23


In [ ]:
pickle.dump(listings, open('all_SF_listings.p','wb'))

In [82]:
listings.shape

(336718, 106)

In [86]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)